# Overview
This notebook illustrates the impact of the latent noise distribution on the resulting error statistics of generated samples

In [1]:
%load_ext autoreload
%autoreload 2
%load_ext wurlitzer
#^^^ the wurlitzer extension is used to capture C/C++ output to be displayed in the notebook
#^^^ this is very useful for debugging, but it doesn't work on windows
import os
os.chdir("../")

In [13]:
import random
from typing import Tuple
import pickle
import time
import yaml

import config
from src import robots
from src.robots import get_robot
from src.ik_solvers import IkflowSolver, GenerativeIKSolver
from src.math_utils import rotation_matrix_from_quaternion, geodesic_distance
from src.utils import get_ik_solver

import numpy as np
from tqdm import tqdm
import matplotlib.pyplot as plt
import torch

from klampt import *
from klampt.math import vectorops, so3, se3
from klampt import vis
from klampt.model.create import primitives
from IPython.display import clear_output
from klampt.vis.ipython import EditConfig,EditPoint,EditTransform

In [14]:
with open("model_descriptions.yaml", "r") as f:
    MODEL_DESCRIPTIONS = yaml.safe_load(f)

In [17]:
pi = np.pi
model_name="panda_tpm"
terrain_filepath = "urdfs/klampt_resources/terrains/plane.off"

In [18]:
# wandb_run_name = TPMS[TPM_NAME]["wandb_run_name"]

def _get_ik_solver(model_name):
    
    model_weights_filepath = MODEL_DESCRIPTIONS[model_name]["model_weights_filepath"]
    robot_name = MODEL_DESCRIPTIONS[model_name]["robot_name"]
    hparams = MODEL_DESCRIPTIONS[model_name]
    # Build GenerativeIKSolver and set weights
    return get_ik_solver(model_weights_filepath, robot_name, hparams)[0]


# robonaut2 -> ik not implemented
# valkyrie -> urdf is all wacky
# atlas_arm -> Only 6dof so don't see a proper distribution

# # PANDA
ik_solver = _get_ik_solver(model_name)
urdf_filepath = "models/panda_arm/panda_gripper_shifted.urdf"
R = so3.from_rpy([0, pi/2, 0])
target_T_iktarget = (R, [-0.1, 0, 0])


# ATLAS
# robot_model, model_wrapper = get_model_wrapper(wandb_run_name)
# urdf_filepath = robot_model.urdf_filepath
# terrain_filepath = "models/klampt_resources/terrains/plane_for_atlas.off"
# R = so3.from_rpy([0, 0, 0])
# target_T_iktarget = (R, [0, -.125, -.01])


# Robosimian
# robot_model = get_robot("robosimian")
# model_wrapper, _ = get_model_wrapper(wandb_run_name)
# urdf_filepath = model_wrapper.robot_model.urdf_filepath
# R = so3.from_rpy([0, 0, 0])
# target_T_iktarget = (R, [0, 0, 0])

# PR2
# robot_model = get_robot("pr2")
# model_wrapper, _ = get_model_wrapper(wandb_run_name)
# urdf_filepath = model_wrapper.robot_model.urdf_filepath
# R = so3.from_rpy([0, 0, 0])
# target_T_iktarget = (R, [0, 0, 0])


# PR2
# robot_model = get_robot("baxter")
# model_wrapper = ModelWrapper_cINN_Softflow(cINN_SoftflowParameters(), robot_model)
# urdf_filepath = model_wrapper.robot_model.urdf_filepath
# R = so3.from_rpy([0, 0, 0])
# target_T_iktarget = (R, [0, 0, 0])
# terrain_filepath = "urdfs/klampt_resources/terrains/plane_for_atlas.off"

# Generic
klampt_robot_model = ik_solver.robot._klampt_robot

WorldModel::LoadRobot: ./urdfs/panda_arm/panda.urdf
URDFParser: Link size: 17
URDFParser: Joint size: 12
LoadAssimp: Loaded model ./urdfs/panda_arm/meshes/collision/link0.stl (595 verts, 200 tris)
LoadAssimp: Loaded model ./urdfs/panda_arm/meshes/collision/link1.stl (887 verts, 300 tris)
LoadAssimp: Loaded model ./urdfs/panda_arm/meshes/collision/link2.stl (889 verts, 300 tris)
LoadAssimp: Loaded model ./urdfs/panda_arm/meshes/collision/link3.stl (900 verts, 300 tris)
LoadAssimp: Loaded model ./urdfs/panda_arm/meshes/collision/link4.stl (900 verts, 300 tris)
LoadAssimp: Loaded model ./urdfs/panda_arm/meshes/collision/link5.stl (900 verts, 300 tris)
LoadAssimp: Loaded model ./urdfs/panda_arm/meshes/collision/link6.stl (600 verts, 200 tris)
LoadAssimp: Loaded model ./urdfs/panda_arm/meshes/collision/link7.stl (600 verts, 200 tris)
LoadAssimp: Loaded model ./urdfs/panda_arm/meshes/collision/hand.stl (600 verts, 200 tris)
LoadAssimp: Loaded model ./urdfs/panda_arm/meshes/collision/finger.s

# Visualizing

Per ieee: "One column width: 3.5 inches, 88.9 millimeters, or 21 picas "

Note: Don't change the color here. The standalone gui has a tool to change the appearance properties of objects in the world

When positioning the camera, try to fill the frame almost entirely with the robot.

In [19]:
SHOW_MUG = True

In [21]:
world = WorldModel()
world.loadTerrain(terrain_filepath)
world.loadRobot(urdf_filepath)
# print(world.numRobots())

if SHOW_MUG:
    world.loadRigidObject("urdfs/klampt_resources/objects/mug_klampt.obj")
    mug = world.rigidObject(0)
robot = world.robot(0)

0WorldModel::LoadRobot: models/panda_arm/panda_gripper_shifted.urdf



URDFParser: RobotModel::LoadURDF: error parsing XML


RuntimeError: Invalid robot index

In [9]:
# Open viz in pop up window.
vis.kill()
vis.init(["PyQt"])
vis.add("world", world)
vis.show()

***  klampt.vis: using Qt5 as the visualization backend  ***
QtBackend: initializing app as Klamp't visualization


vis: creating GL window
######### QGLWidget setProgram ###############
#########################################
klampt.vis: Making window 0
#########################################
######### QGLWidget Initialize GL ###############
TriMeshTopology: mesh has 10 triangles with duplicate neighbors!
  Triangle range 0 to 3
  May see strange results for some triangle mesh operations
CreaseMesh: Invalid triangle 0 2 -1, input triangle 0 1 1
CreaseMesh: Invalid triangle 11 7 -1, input triangle 3 2 2
CreaseMesh: Invalid triangle 4 -1 8, input triangle 1 1 2
CreaseMesh: Invalid triangle 5 10 -1, input triangle 1 2 2
TriMeshTopology: mesh has 10 triangles with duplicate neighbors!
  Triangle range 0 to 3
  May see strange results for some triangle mesh operations


In [10]:
# Tf parameters
R = so3.from_rpy([pi/2, 0, -0.3])
target_T_mug = (R, [0.065, 0.005, -0.005])

# Robot parameters
alpha = 1
robot_color = (1, 1, 1, alpha)

def update_sols(_end_pose):
    print(f"_end_pose = {_end_pose.tolist()}")
    if SOL_MODE == "IKFLOW":
        sols, _ = model_wrapper.make_samples(_end_pose, N_SOLUTIONS)    
#         sols = np.load("/home/jeremysmorgan/Projects/nn_ik_softflow2/atlas_samples.npy")
    elif SOL_MODE == "IK":
        sols = model_wrapper.robot_model.inverse_kinematics(_end_pose, N_SOLUTIONS, debug=False)
    else:
        raise ValueError("Unknown mode")

    qs = robot_model.x_to_qs(sols)
    for i in range(N_SOLUTIONS):
        q = qs[i]
        if i == 0:
            vis.setColor(vis.getItemName(robot), robot_color[0], robot_color[1], robot_color[2], a=robot_color[3])
            robot.setConfig(q)
            continue
        vis.add(f"robot_{i}", q, color=robot_color)
        
def update_scene():
    target = vis.getItemConfig("target")
    world_T_target = (target[:-3], target[-3:])   
    print("world_T_target =", world_T_target)
    if SHOW_MUG:
        world_T_mug = se3.mul(world_T_target, target_T_mug)
        mug.setTransform(world_T_mug[0], world_T_mug[1])
    world_T_iktarget = se3.mul(world_T_target, target_T_iktarget)
    update_sols(np.array(world_T_iktarget[1] + so3.quaternion(world_T_iktarget[0])))
    vis.update()
    
def current_state():
    target = vis.getItemConfig("target")
    world_T_target = (target[:-3], target[-3:])   
    print("world_T_target =", world_T_target)
    print("target_T_iktarget =", target_T_iktarget)
    vp = vis.getViewport()
    print("vp = vis.getViewport()")
    print("vp.camera.rot =", vp.camera.rot)
    print("vp.camera.tgt =", vp.camera.tgt)
    print("vp.camera.dist =", vp.camera.dist)
    print("vp.clippingplanes = ", vp.clippingplanes)
    print("vis.setViewport(vp)")

CreaseMesh: Invalid triangle 0 2 -1, input triangle 0 1 1
CreaseMesh: Invalid triangle 11 7 -1, input triangle 3 2 2
CreaseMesh: Invalid triangle 4 -1 8, input triangle 1 1 2
CreaseMesh: Invalid triangle 5 10 -1, input triangle 1 2 2


In [11]:
def print_mmd(_end_pose, k=30, n_solutions=100, n_terms = 1):
    scores = []
    scores_random = []
    scores_ikik = []
    
    for i in range(k):
        sols_ikflow, _ = model_wrapper.make_samples(_end_pose, n_solutions)  
        
        ee_pose_ikflow = model_wrapper.robot_model.forward_kinematics(sols_ikflow)
        ave_l2err = np.mean(np.linalg.norm(ee_pose_ikflow[:, 0:3] - _end_pose[0:3], axis=1))
        
        sols_ik = model_wrapper.robot_model.inverse_kinematics(_end_pose, n_solutions, debug=False)
        scores.append(mmd(sols_ikflow, sols_ik, n_terms=n_terms))
        
        sols_ik2 = model_wrapper.robot_model.inverse_kinematics(_end_pose, n_solutions, debug=False)
        scores_ikik.append(mmd(sols_ik, sols_ik2, n_terms=n_terms))
        
        sols_random = model_wrapper.robot_model.sample(n_solutions)
        scores_random.append(mmd(sols_random, sols_ik, n_terms=n_terms))
        
    print(f"\nprint_mmd(k={k}, n_terms={n_terms}, n_solutions={n_solutions})")
#     print("\n mean ee.err (mm):", round(100*ave_l2err, 4))
    print("     (random)    mmd:", np.mean(scores_random), "\tstd:", np.std(scores_random))
    print("     (ik-ik)     mmd:", np.mean(scores_ikik), "\tstd:", np.std(scores_ikik))
    print("     (ikflow-ik) mmd:", np.mean(scores), "\tstd:", np.std(scores))

def print_current_mmd(k=1, n_solutions=1000, n_terms = 1):
    target = vis.getItemConfig("target")
    world_T_target = (target[:-3], target[-3:])   
    world_T_iktarget = se3.mul(world_T_target, target_T_iktarget)
    pose = np.array(world_T_iktarget[1] + so3.quaternion(world_T_iktarget[0]))
    print_mmd(pose, k=k, n_solutions=n_solutions, n_terms = n_terms)

### Set IK target

In [12]:
# Update the target to a particular pose


# -- Panda
# pose = [ 0.50, 0, 0.65,  1, 0, 0, 0]
# world_T_target = ([1.0, 0, 0, 0, 1.0, 0, 0, 0, 1.0], [0.35, 0, 0.65])
# vis.add("target", world_T_target, length=.25, width=1)

# -- 
pose = [ 0.50, 0, 0.65,  1, 0, 0, 0]
world_T_target = ([1.0, 0, 0, 0, 1.0, 0, 0, 0, 1.0], [0.35, 0, 0.65])
vis.add("target", world_T_target, length=.25, width=1)


# -- Atlas
# pose = [1.24028463e-01,  6.72724128e-01,  2.65556256e-01,  7.02780096e-01, 0.00000000e+00,  1.11022302e-16, -7.11407152e-01]
# vis.add("target", (so3.from_quaternion(pose[3:]), pose[0:3]), length=.25, width=1)

# -- Atlas
# pose = [ 0.4, 0.15, 1.25,  -0.518, -.316, 0.318, -.729]
# vis.add("target", (so3.from_quaternion(pose[3:]), pose[0:3]), length=.25, width=1)


vis.hideLabel("target")
vis.update()

In [13]:
vis.edit("target", doedit=True)
vis.setAttribute("target", "length", .2)
vis.hideLabel("target")

In [ ]:
vis.edit("target", doedit=False)
vis.setAttribute("target", "length", .2)
vis.hideLabel("target")

### Update scene

In [63]:
SOL_MODE = "IK"
# SOL_MODE = "IKFLOW"
N_SOLUTIONS = 10

update_scene()

world_T_target = ([-0.2137132837313067, 0.9370873236417957, -0.2760325709524874, -0.029779749730973304, 0.27618005340430996, 0.9606444423445929, 0.9764424196350465, 0.21352265915225443, -0.03111712014000312], [1.0190904590701242, 0.6326763495116817, 0.24953221839773818])
_end_pose = [1.0190904590701242, 0.6326763495116817, 0.24953221839773818, 0.5077769317163291, 0.3678395652333999, 0.6166462635246446, 0.47602943978996576]


### Printout the current mmd


In [ ]:
print_current_mmd()

### Printout the current vis. state

In [ ]:
current_state()

## Replicate visualizations used in the papaer:


### Panda cover photo

``` python
world_T_target = ([1.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0], [0.44061820812564334, 0.0, 0.6702390809215747])
target_T_iktarget = ([6.123233995736766e-17, 0.0, -1.0, 0.0, 1.0, 0.0, 1.0, 0.0, 6.123233995736766e-17], [-0.1, 0, 0])
vp = vis.getViewport()
vp.camera.rot = [0.0, -0.3500000000000001, 2.3699999999999948]
vp.camera.tgt = [0.1510013083207166, 0.015973609866733036, 0.4659988966788869]
vp.camera.dist = 1.807165271473198
vp.clippingplanes =  (0.2, 20)
vis.setViewport(vp)
```

### IK vs IKFlow comparison


##### PandaArm:

``` python
world_T_target = ([-0.010553079379030494, -2.7755575615628914e-16, 0.9999443147073834, 0.9929081491431708, -0.11842128885441006, 0.010478822030265622, 0.1184146945302888, 0.9929634426030292, 0.0012497092614485172], [-0.020075173440280116, 0.48281258570362073, 0.723629099140075])
target_T_iktarget = ([6.123233995736766e-17, 0.0, -1.0, 0.0, 1.0, 0.0, 1.0, 0.0, 6.123233995736766e-17], [-0.1, 0, 0])
vp = vis.getViewport()
vp.camera.rot = [0.0, -0.7100000000000004, 4.349999999999992]
vp.camera.tgt = [0.04346345314554039, 0.24269145260400948, 0.4130087064101574]
vp.camera.dist = 2.4394179584435967
vp.clippingplanes =  (0.2, 20)
vis.setViewport(vp)
```

print_mmd(k=1, n_terms=1, n_solutions=1000)
     (random)    mmd: 0.04902878776192665 	std: 0.0
     (ik-ik)     mmd: 0.0011280993930995464 	std: 0.0
     (ikflow-ik) mmd: 0.012286311946809292 	std: 0.0
    
##### Atlas:

``` python
world_T_target = ([-0.012200271833502363, -0.9999255745781075, -1.5604852833744659e-16, 0.9999255745781075, -0.012200271833502363, -1.5796411934860783e-16, 1.5604852833744659e-16, -1.5796411934860783e-16, 1.0], [0.24901916, 0.671199094, 0.275556256])
pose = np.array([ 2.49019160e-01,  6.71199094e-01,  2.75556256e-01,  7.02780096e-01,
        0.00000000e+00,  1.11022302e-16, -7.11407152e-01])
vp = vis.getViewport()
vp.camera.rot = [0.0, -0.23, 2.2600000000000033]
vp.camera.tgt = [0.012435107852541857, -0.050844730904928005, 0.06010287306824789]
vp.camera.dist = 5.429024508215758
vp.clippingplanes =  (0.2, 20)
vis.setViewport(vp)
```

print_mmd(k=1, n_terms=1, n_solutions=1000)
     (random)    mmd: 0.09190898388624191 	std: 0.0
     (ik-ik)     mmd: 0.0003174212761223316 	std: 0.0
     (ikflow-ik) mmd: 0.008037609048187733 	std: 0.0